In [1]:
"""
    Redmine ticket 6965 - Sec aims seen for follow ups - due to incosistency in cohorts after reconsenting
"""
# Imports
import json
import os
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
"""
   Identify cohort inconsistencies 
"""
child_consents = CaregiverChildConsent.objects.values_list('subject_identifier', flat=True)
child_consents = set(list(child_consents))

inconsistencies = {}
for pid in child_consents:
    cohorts = CaregiverChildConsent.objects.filter(
        subject_identifier=pid).values_list('cohort', flat=True).distinct()
    cohorts = list(set(cohorts))
    if len(cohorts) > 1:
        inconsistencies[pid] = cohorts
print(len(inconsistencies))

for pid, cohorts in inconsistencies.items():
    for cohort in cohorts:
        consent = CaregiverChildConsent.objects.filter(
            subject_identifier=pid, cohort=cohort).earliest('consent_datetime')
        print('pid:', pid, ', cohort:', cohort, ', version:',
              consent.version, ', consent_date:', consent.consent_datetime.date())

inconsistencies

9
pid: B142-040990116-1-60 , cohort: cohort_b , version: 1 , consent_date: 2021-06-25
pid: B142-040990116-1-60 , cohort: cohort_b_sec , version: 3 , consent_date: 2023-03-10
pid: B142-040991010-5-10 , cohort: cohort_b_sec , version: 2.1 , consent_date: 2022-10-21
pid: B142-040991010-5-10 , cohort: cohort_c_sec , version: 3 , consent_date: 2023-01-26
pid: B142-040990272-2-10 , cohort: cohort_c , version: 1 , consent_date: 2021-07-29
pid: B142-040990272-2-10 , cohort: cohort_c_sec , version: 3 , consent_date: 2023-05-03
pid: B142-040990159-1-10 , cohort: cohort_c , version: 3 , consent_date: 2023-04-17
pid: B142-040990159-1-10 , cohort: cohort_c_sec , version: 1 , consent_date: 2021-07-06
pid: B142-040990172-4-10 , cohort: cohort_c , version: 2.1 , consent_date: 2022-08-26
pid: B142-040990172-4-10 , cohort: cohort_c_sec , version: 1 , consent_date: 2021-07-08
pid: B142-040990562-6-10 , cohort: cohort_b_sec , version: 3 , consent_date: 2023-02-28
pid: B142-040990562-6-10 , cohort: cohort_

{'B142-040990116-1-60': ['cohort_b', 'cohort_b_sec'],
 'B142-040991010-5-10': ['cohort_b_sec', 'cohort_c_sec'],
 'B142-040990272-2-10': ['cohort_c', 'cohort_c_sec'],
 'B142-040990159-1-10': ['cohort_c', 'cohort_c_sec'],
 'B142-040990172-4-10': ['cohort_c', 'cohort_c_sec'],
 'B142-040990562-6-10': ['cohort_b_sec', 'cohort_a'],
 'B142-040990161-7-10': ['cohort_c', 'cohort_c_sec'],
 'B142-040990096-5-25': ['cohort_c', 'cohort_b'],
 'B142-040990096-5-35': ['cohort_c', 'cohort_b']}

In [3]:
"""
    Check secondary aims participant's scheduled for followups
"""

sec_aims = CaregiverChildConsent.objects.filter(
    cohort__endswith='_sec').values_list('subject_identifier', flat=True).distinct()
sec_aims = list(set(sec_aims))
print('secondary aims:', len(sec_aims))

sec_fu_schedule = ParticipantNote.objects.filter(title='Follow Up Schedule', subject_identifier__in=sec_aims)
print('scheduled for FU', sec_fu_schedule.count())

# Check if pids exist in the inconsistencies
sec_fu_schedule = sec_fu_schedule.values_list('subject_identifier', flat=True).distinct()
exists = []
for pid in sec_fu_schedule:
    exists.append(pid in inconsistencies.keys())
print(sec_fu_schedule)
print(all(exists))

fu_sec_aims = SubjectScheduleHistory.objects.filter(
    subject_identifier__in=sec_aims, schedule_name__icontains='_fu')
fu_sec_aims.count()

print(sec_fu_schedule)

secondary aims: 469
scheduled for FU 4
<QuerySet ['B142-040990159-1-10', 'B142-040990272-2-10', 'B142-040990161-7-10', 'B142-040990172-4-10']>
True
<QuerySet ['B142-040990159-1-10', 'B142-040990272-2-10', 'B142-040990161-7-10', 'B142-040990172-4-10']>


In [4]:
"""
    Identify recalculated cohort and onschedules to confirm re-enrolment
"""

import copy
from flourish_caregiver.helper_classes.onschedule_helper import OnScheduleHelper

cohort_inc = copy.deepcopy(inconsistencies)

caregiver_models = dir(django_apps.get_app_config('flourish_caregiver').models_module.onschedule)
child_models = dir(django_apps.get_app_config('flourish_child').models_module.onschedule)

recalculated = {}
caregiver_onschedules = {}
child_onschedules = {}
for pid, cohorts in cohort_inc.items():
    helper_cls = OnScheduleHelper(pid)
    earliest_consent = CaregiverChildConsent.objects.filter(subject_identifier=pid).earliest('consent_datetime')

    cohorts.remove(earliest_consent.cohort)

    print('pid', pid, 'recalculated', earliest_consent.cohort, '->', cohorts)
    recalculated[pid] = cohorts

    for cohort in cohorts:
        cohort = ''.join(cohort.split('_'))
        if 'sec' in cohort:
            caregiver_onschedules[pid] = [model for model in caregiver_models if cohort in model.lower()]
            child_onschedules[pid] = [model.lower() for model in child_models if cohort in model.lower()]
        else:
            caregiver_onschedules[pid] = [model.lower() for model in caregiver_models if cohort in model.lower() and 'sec' not in model.lower()]
            child_onschedules[pid] = [model.lower() for model in child_models if cohort in model.lower() and 'sec' not in model.lower()]


pid B142-040990116-1-60 recalculated cohort_b -> ['cohort_b_sec']
pid B142-040991010-5-10 recalculated cohort_b_sec -> ['cohort_c_sec']
pid B142-040990272-2-10 recalculated cohort_c -> ['cohort_c_sec']
pid B142-040990159-1-10 recalculated cohort_c_sec -> ['cohort_c']
pid B142-040990172-4-10 recalculated cohort_c_sec -> ['cohort_c']
pid B142-040990562-6-10 recalculated cohort_a -> ['cohort_b_sec']
pid B142-040990161-7-10 recalculated cohort_c -> ['cohort_c_sec']
pid B142-040990096-5-25 recalculated cohort_b -> ['cohort_c']
pid B142-040990096-5-35 recalculated cohort_b -> ['cohort_c']


In [117]:
from django.db.models.deletion import ProtectedError
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment
"""
    1. Check child enrolments and remove onschedule, subject history, appointments associated
    with cohort, unless visits already capured.
    2. Check caregiver enrolments and follow steps above
    3. Correct cohort inconsistency
"""

def correct_inconsistency(data, app_label, appointment_cls=None, visit_cls=None, query_field=None):
    for pid, onschedules in data.items():
        for onschedule in onschedules:
            model_cls = django_apps.get_model(f'{app_label}.{onschedule}')
            try:
                onsch_obj = model_cls.objects.get(**{f'{query_field}': pid})
            except model_cls.DoesNotExist:
                continue
            else:
                # Check if appointment/visit(s) captured
                appts = appointment_cls.objects.filter(subject_identifier=onsch_obj.subject_identifier,
                                                       schedule_name=onsch_obj.schedule_name, )
                visits = visit_cls.objects.filter(appointment__in=appts.values_list('id', flat=True))
                # If visit(s) already captured don't remove anything
                if visits.exists():
                    print('It has visits, disapointing 😞', pid)
                    continue
                # Otherwise remove
                print('Correcting...', pid)
                try:
                    appts.delete()
                except ProtectedError:
                    print('Someone has a secret relationship 😒', pid)
                    raise
                else:
                    schedule_history = SubjectScheduleHistory.objects.filter(
                        subject_identifier=pid,
                        schedule_name=onsch_obj.schedule_name, )
                    schedule_history.delete()
                    onsch_obj.delete()
                    print('Done', pid)


In [118]:
# Correct for child
correct_inconsistency(child_onschedules,
                      'flourish_child',
                      appointment_cls=ChildAppointment,
                      visit_cls=ChildVisit,
                      query_field='subject_identifier')


Corrected B142-040990272-2-10
Done B142-040990272-2-10
Corrected B142-040990159-1-10
Done B142-040990159-1-10
Corrected B142-040990562-6-10
Done B142-040990562-6-10
It has visits, disapointing 😞 B142-040990096-5-35
Corrected B142-040990096-5-35
Done B142-040990096-5-35
Corrected B142-040990172-4-10
Done B142-040990172-4-10
Corrected B142-040990161-7-10
Done B142-040990161-7-10
Corrected B142-040990116-1-60
Done B142-040990116-1-60
It has visits, disapointing 😞 B142-040990096-5-25
Corrected B142-040990096-5-25
Done B142-040990096-5-25


In [119]:
# Correct for caregiver
correct_inconsistency(caregiver_onschedules,
                      'flourish_caregiver',
                      appointment_cls=Appointment,
                      visit_cls=MaternalVisit,
                      query_field='child_subject_identifier')

Corrected B142-040991010-5-10
Done B142-040991010-5-10
Corrected B142-040990172-4-10
Done B142-040990172-4-10
Corrected B142-040990096-5-25
Done B142-040990096-5-25


In [8]:
# Correct cohort variable
for pid, cohorts in recalculated.items():
    consents = CaregiverChildConsent.objects.filter(subject_identifier=pid, cohort__in=cohorts)
    for consent in consents:
        try:
            dummy_consent = ChildDummySubjectConsent.objects.get(subject_identifier=pid, version=consent.version, )
        except ChildDummySubjectConsent.DoesNotExist:
            print('Impossible... 😳', pid, consent.version)
        else:
            dummy_consent.cohort = None
            dummy_consent.save_base(raw=True)
        consent.cohort = None
        consent.save()
    print('Fixed', pid)


Impossible... 😳 B142-040991010-5-10 3
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(364)caregiver_child_consent_on_post_save()
    362             child_age = age(instance.child_dob, get_utcnow())
    363         import pdb;pdb.set_trace()
--> 364         if not instance.cohort:
    365 
    366             cohort = cohort_assigned(instance.study_child_identifier,

ipdb> instance.cohort
'cohort_b_sec'
ipdb> n
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(393)caregiver_child_consent_on_post_save()
    391         else:
    392             # TO-DO: Update child cohort
--> 393             try:
    394                 prev_enrolled_obj = CaregiverPreviouslyEnrolled.objects.get(
    395                     subject_identifier=instance.subject_consent.subject_identifier)

ipdb> n
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(394)caregiver_child_consent_on_post_save()
    392             # TO-DO: Update c

ipdb> n
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(409)caregiver_child_consent_on_post_save()
    407 
    408                     try:
--> 409                         child_dummy_consent = child_dummy_consent_cls.objects.get(
    410                             subject_identifier=instance.subject_identifier,
    411                             version=instance.version,

ipdb> n
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(410)caregiver_child_consent_on_post_save()
    408                     try:
    409                         child_dummy_consent = child_dummy_consent_cls.objects.get(
--> 410                             subject_identifier=instance.subject_identifier,
    411                             version=instance.version,
    412                             identity=instance.identity)

ipdb> n
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(411)caregiver_child_consent_on_post_save()
    

ipdb> n
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(427)caregiver_child_consent_on_post_save()
    425             update_maternal_dataset_and_worklist(
    426                 instance.subject_consent.subject_identifier,
--> 427                 screening_identifier=instance.subject_consent.screening_identifier,
    428                 study_child_identifier=instance.study_child_identifier)
    429 

ipdb> n
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(428)caregiver_child_consent_on_post_save()
    426                 instance.subject_consent.subject_identifier,
    427                 screening_identifier=instance.subject_consent.screening_identifier,
--> 428                 study_child_identifier=instance.study_child_identifier)
    429 
    430 

ipdb> n
--Return--
None
> /Users/Ame/source/flourish-caregiver/flourish_caregiver/models/signals.py(428)caregiver_child_consent_on_post_save()
    426                 instance.s

ipdb> c
